This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [7]:
import pandas as pd

N=1000000

file_name = "C:/Users/User/Downloads/data_000637.txt"
data = pd.read_csv(file_name,nrows=N) #if N is bigger than the maximum number of rows this create the DataFrame with the maximum row
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
999995,1,0,62,3869209011,2560,29
999996,1,0,51,3869209011,2561,12
999997,1,0,61,3869209011,2562,2
999998,1,0,50,3869209011,2563,4


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [8]:
x=data['BX_COUNTER'].max()+1
print("x=",x)

x= 3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [9]:
number_of_orbit=(data['ORBIT_CNT'].values[-1]-data['ORBIT_CNT'].values[0])
d=number_of_orbit*x*25/10**9
print("duration (s) = ",d)

duration (s) =  0.7880004


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [19]:
t0=data.at[0,'ORBIT_CNT']*x*25+data.at[0,'BX_COUNTER']*25+data.at[0,'TDC_MEAS']*(25/30)
data['time in ns'] = data['ORBIT_CNT']*x*25+data['BX_COUNTER']*25+data['TDC_MEAS']*(25/30)-t0
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time in ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.125000e-01
2,1,0,63,3869200167,2553,28,4.476625e+03
3,1,0,64,3869200167,2558,19,4.594125e+03
4,1,0,64,3869200167,2760,25,9.649125e+03
...,...,...,...,...,...,...,...
999995,1,0,62,3869209011,2560,29,7.880051e+08
999996,1,0,51,3869209011,2561,12,7.880051e+08
999997,1,0,61,3869209011,2562,2,7.880051e+08
999998,1,0,50,3869209011,2563,4,7.880051e+08


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [16]:
grpbyTDC=data.groupby('TDC_CHANNEL').count()
print("noisiest channels:",grpbyTDC.sort_values(by=['TDC_MEAS'],ascending = False)[0:3].index.values)

noisiest channels: [139  64  63]


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [17]:
grpbyORBIT=data.groupby('ORBIT_CNT')
print("number of non empty orbits : ",len(grpbyORBIT.count()))
TDC139=data[data['TDC_CHANNEL']==139]
grpbyORBIT139=TDC139.groupby('ORBIT_CNT')
print("number of unique empty orbits  with at least one measurement from TDC_CHANNEL=139: ",len(grpbyORBIT139.count()))

number of non empty orbits :  8841
number of unique empty orbits  with at least one measurement from TDC_CHANNEL=139:  8816


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel